In [ ]:
from tkinter import *
from PIL import Image, ImageTk
from tkinter import messagebox
import math
import numpy as np
import csv
import time
import tkinter.font as font
import os.path

# paths for files
# scenario_path = "/home/sara/Notebook_Script/Data_Scenarios/"
# logfile_path = "/home/sara/Notebook_Script/"
scenario_path = "/home/resl/human_auv_pp_scenarios/"
logfile_path = "/home/resl/human_auv_pp_userfiles/"

#stands for circle size, this determines the size of the canvas and plot as well
cs = 30

#stands for font size
fs = 25

#start point x and y coords
last_x = int(cs/2)
last_y = int(cs/2)

#number of circles deleted/points recorded
delete_count = 0

#number of plots for the user to create paths in
max_plots = 12

#create the root
master = Tk()
master.title("Data Path Creator")

#create a new font
global my_font
my_font = font.Font (master,"Times " + str(fs))

#set the message box font
master.option_add('*font', "Times " + str(fs))

#overlays a grid of black circles over the photo in the canvas w
def create_grid(plot_name):
    #create the image and add it to the canvas
    image = Image.open(plot_name)
    photo = ImageTk.PhotoImage(image.resize((cs*arr_width+int((8*cs)/10),cs*arr_height+12)),Image.ANTIALIAS)
    #keep a reference of the image
    w.image = photo
    w.create_image((arr_width*cs)/2,(arr_height*cs)/2,image=photo, state=NORMAL)
    
    #create the array or circles
    global circle_arr
    circle_arr = []
    for r in range (0,arr_height):
        for c in range (0,arr_width):
#             circle_arr.append(w.create_oval((cs*c,cs*r,cs*c+cs,cs*r+cs),activefill= "red",fill="black", width = int(cs/2.5)))
            circle_arr.append(w.create_rectangle((cs*c,cs*r,cs*c+cs,cs*r+cs),activefill="red",fill="black"))
    circle_arr = np.reshape(np.asarray(circle_arr),(arr_height,arr_width))
    
    w.pack(side = LEFT, anchor = 'e', padx = 15)
    colorbar.pack(side = LEFT, anchor = 'w')
    
#deletes the circles on the line between the current point and the last point,
#and keeps track of the number of circles deleted
def callback(event):
    #calculate center of clicked circle (in pixels)
    centered_x = event.x + int(cs/2) - (event.x%cs)
    centered_y = event.y + int(cs/2) - (event.y%cs)
    
    #make sure that the global last_x and last_y, not the local versions, are being modified
    global last_x
    global last_y
    
    # draw line to clicked circle (can use this to check if circles on line are being deleted)
    w.create_line(last_x,last_y,centered_x,centered_y, fill = "red")
    
    #calculate the min and max of the line so that the for loop goes from smallest to largest value
    min_x = min(last_x, centered_x)
    min_y = min(last_y, centered_y)
    max_x = max(last_x, centered_x)
    max_y = max(last_y, centered_y)
    
    #make sure that the global delete_count, not the local version, is modified
    global delete_count

    #calculate slope using delta y/delta x
    if(centered_x != last_x):
        slope = float((centered_y - last_y) / (centered_x - last_x))
    
    #check to see if should loop through x or y
    #want to loop through whichever has a bigger change because then every circle on the line gets deleted
    #first case: change in x larger than change in y
    if(max_x-min_x > max_y-min_y):
        #find the y value that corresponds with the smallest x value (in pixels)
        #this is so the point (min_x, corresp_y) can be used as (x1, y1) in the point-slope equation below
        if (min_x == last_x):
            corresp_y = last_y
        else:
            corresp_y = centered_y
            
        #decide whether to increment up or down in the for loop (if last_x equals min_x increment up, otherwise down)
        if (last_x != min_x):
            increment = -int(cs)
        else:
            increment = int(cs)
        
        #loop through the x values in pixels
        for x in range (last_x,centered_x+increment,increment):
            #calculate the y coordinate in pixels based off of slope, the x coord, and the min point
            #this uses point-slope form: y-y1 = m(x-x1)
            y = (slope * (x - min_x) + corresp_y)
            
            delete(x,y)
            
    #second case: change in y greater than or equal to change in x
    else:
        #find the x value that corresponds to the smallest y value (in pixels)
        #this is so this point (corresp_x, min_y) can be used as (x1,y1) in the point-slope equation below
        if (min_y == last_y):
            corresp_x = last_x
        else:
            corresp_x = centered_x
        
        increment = int(cs)
        if (last_y != min_y):
            increment = -int(cs)
            
        #loop through the y values (in pixels)
        for y in range (last_y,centered_y+increment,increment):
            #checking to see that delta x is not zero (so can calculate m without error)
            if(centered_x != last_x):                
                #use the point-slope equation of the line to solve for the x coordinate in pixels based off of the y,
                #slope, and a point (x1, y1) [(y-y1)/m = x-x1]
                x = ((y - min_y) / slope) + corresp_x
                delete(x,y)
                    
            #handle the case where delta x is zero to avoid division by zero error
            else:
                #loop through each circle on the line by incrementing by the diameter of the circle, in pixels
                for y_count in range (min_y, max_y+1,cs):
                    x = centered_x
                    delete(x,y_count)
                #break to avoid going through the outer for loop
                break
    
    # update the last clicked location
    last_y = centered_y
    last_x = centered_x
    
    #change the text in the labels that show the delete count and the distance remaining in order to reflect the new
    #values for these variables
    delete_counter.config(text = "Total Points Recorded:\n" + str(delete_count))
    total_left.config(text = "Total Distance Remaining:\n" + str(max_circ-delete_count))
    
    #if there is no distance left, display a warning box
    if (max_circ <= delete_count):
        messagebox.showwarning(title = "No Distance Left", message = "No distance left in plot " + str(plot_number) + "/12. Save or quit.")
    #if there is a low distance left, turn the text for Total Distance Remaining to red
    if (max_circ-delete_count < 50):
        total_left.config(fg = "red")
    
#saves the data values that correspond to the deleted circles to file
def save_callback():
    #initializing the x, y, and data arrays
    x_vals = []
    y_vals = []    
    data_vals = []
    
    #read the file and add the x, y, and data values to their respective arrays
    global plot_number, scenario_path
    with open (scenario_path + 'field_' + str(plot_number) + '.csv') as fl:
        reader = csv.reader(fl, delimiter = ',')
        index = 1
        for row in reader:
            #check that the data, and not the header lines, are being read
            if index >= 4:
                x_vals.append(float(row[0]))
                y_vals.append(float(row[1]))
                data_vals.append(float(row[2]))
            else:
                index+=1
                
    #reshape the x,y,and data arrays to reflect circle_arr and the points in the image
    x_vals = np.asarray(x_vals).reshape(arr_height,arr_width)
    y_vals = np.asarray(y_vals).reshape(arr_height,arr_width)  
    data_vals = np.asarray(data_vals).reshape(arr_height,arr_width)
    
    #only request the user name after the first plot
    #so the user only has to input their name once
    if (plot_number == 1):
        name = ask_for_name(master)
    
    #check to see that cancel button was not pushed
    if user_name != "&#*%":
        file_name = time.strftime (user_name+'_path_'+str(plot_number)+'.csv')
        
        #make a new folder
        global logfile_path
        newpath = logfile_path+user_name 
        if not os.path.exists(newpath):
            os.makedirs(newpath)

        #open the file and write the header
        file = open (logfile_path+user_name+'/'+file_name, 'w')
        file.write ('Longitude,Latitude,Total Water Column (m)\n')

        #loop through the indices of the arrays
        for r in range (0,arr_height):
            for c in range (0,arr_width):
                #if the corresponding value in circle_arr has been "recorded" (deleted),
                #then add its x,y, and data value to the file so the values for the full path are recorded
                if is_deleted(r,c):
                    file.write(str(x_vals[arr_height-1-r][c]) + ',' + str(y_vals[arr_height-1-r][c]) + ',' + str(data_vals[arr_height-1-r][c]) + '\n')
            
        #display a message box to show that the file has been saved, and show the name it is saved under
        if (plot_number == max_plots):
            added_string = "All plots completed."
            #messagebox.showinfo(title = "File Saved", message = "File saved successfully as: " + file_name + added_string)
            messagebox.showinfo(message = added_string)
            
        #if there are no plots left, close the window
        if plot_number < max_plots:
            plot_number += 1

            #delete everything from the canvas
            w.delete("all")

            #create a new grid
            create_grid (scenario_path+str(plot_number)+"_field.png")

            #reset all variables
            global delete_count
            delete_count = 0

            global last_x,last_y
            last_x = int(cs/2)
            last_y = int(cs/2)

            #reset the values displayed on the buttons
            delete_counter.config(text = "Total Points Recorded:\n" + str(0))
            total_left.config(text = "Total Points Remaining:\n" + str(max_circ), fg = "black")

        else:
            #close the window
            global frame
            master.destroy()

    
#return whether the circle at (row, col) has been tagged as deleted or not
def is_deleted(row, col):
    #make an array of the circles tagged as deleted
    global w
    global frame
    w.create_polygon (0,0,50,50)
    deleted_arr = np.asarray(w.find_withtag("deleted"))
    #loop through the array, return true if the circle at (row, col) is in the array and false otherwise
    for del_circle in deleted_arr:
        if (circle_arr[row][col]==del_circle):
            return True
    return False

#deletes the circle at the given coordinates (x,y)
def delete (x,y):
    #calculate the index in circle_arr of the circle at (x,y)
    c_circle = int((x-(x%cs))/cs)
    r_circle = int((y-(y%cs))/cs)
    global delete_count
    
    #tag the clicked circle as deleted, make it transparent, and increment the num of circles deleted
    #only do this if the circle has not already been deleted and there is still distance left
    if is_deleted(r_circle,c_circle)==False and (max_circ-delete_count>0):
        w.itemconfig(circle_arr[r_circle][c_circle], fill = "", tag = "deleted", width = 0)
        delete_count+=1

#asks for the user name, so the files can be more easily separated/categorized
def ask_for_name(master):
    #create a new window
    global window
    window = Toplevel(master)
    
    #create the entry box and text in the new window
    Label(window, text="User Name:").pack()
    global e1
    e1 = Entry(window, font = my_font)
    e1.pack()
    
    #bind the ok_callback1 method to hitting enter (so the user can save by hitting enter)
    e1.bind("<Return>",ok_callback1)
    
    #bind clicking the ok button to the ok_callback method (the user can save by clicking the button)
    b = Button(window, text="OK", command=ok_callback)
    b.pack()
    
    #create button to cancel out of save window
    cancel_button = Button (window, text="Cancel", command = cancel_callback)
    cancel_button.pack()
    
    #update the tasks that were running in the background 
    # this allows for the user name to be retrieved from the entry widget
    master.update_idletasks()
    
    #wait for the window to close before continuing
    #this ensures that the contents of the entry box will be stored in file_name
    master.wait_window(window = window)
    
    #save the name that was entered in the variable user_name
    global user_name
    return user_name

#saves the contents of the entry widget to user_name and closes the window
def ok_callback():
    #storing contents of entry widget to user_name
    global e1
    global user_name
    user_name = e1.get()
    
    #close the window
    global window
    window.destroy()
    
#calls the other ok_callback method
#this is written because the return key needs to be bound to a method with one parameter, and the ok button needs to 
#be bound to a method with zero parameters
def ok_callback1(event):
    ok_callback()
    
#closes the window without saving
def cancel_callback():
    global window
    window.destroy()
    global user_name
    user_name = "&#*%"

#create a frame for the whole window
global frame
frame = Frame (master)
frame.pack()

#make a text box to instruct the user
T = Text (frame, height = 5,  font = my_font)
T.tag_configure("center", justify = 'center')
T.insert(END, "The robot starts in the top left corner. Click on a point in order to reveal\n the data on the path in between it and the previous point, with the goal of\ncollecting the data that will give the best possible model of algae\ndistribution. The maximum length of the entire path is 190 points, and\nthe total points remaining are shown below.", "center")
T.config(state=DISABLED)
T.pack(side = TOP)

#create a frame for the big canvas, colorbar, and colorbar labels
img_frame = Frame (frame)
img_frame.pack()

#set the height and width of the array of data points
global arr_width, arr_height
arr_width = 41
arr_height = 21

#create a canvas
global w
w = Canvas (img_frame, width = (cs*arr_width), height = (cs*arr_height))

#create a colorbar
colorbar = Canvas (img_frame, width = int(4.5*cs), height = (cs*arr_height-10))
img = Image.open(scenario_path + 'colorbar.png')
photo_cbar = ImageTk.PhotoImage(img.resize((int(4.5*cs),cs*arr_height),Image.ANTIALIAS))
#keep a reference of the image
colorbar.image = photo_cbar
colorbar.create_image(int(2.25*cs),(arr_height*cs)/2,image=photo_cbar, state=NORMAL)

#change font size
#font.nametofont(my_font).configure(size=fs)

#create a variable that stores the plot number so the plot changes when the file is saved
global plot_number
plot_number = 1

#method fully defined above, create black grid of black circles over image
arr_width = 41
arr_height = 21                                                                                                                                
create_grid(scenario_path+str(plot_number)+"_field.png")

#create a frame for the colorbar labels
cbar_frame = Frame (img_frame)
cbar_frame.pack(side = LEFT, anchor = 'e')

#create labels for the colorbar
top_label = Label (cbar_frame, text = "-High Algae Content\n \n \n \n \n \n \n \n \n", anchor = 'w', font = my_font)
top_label.pack(side = TOP, anchor = 'w')
bot_label = Label (cbar_frame, text = "\n \n \n \n \n \n-Low Algae Content", anchor = 'w', font = my_font)
bot_label.pack(side = BOTTOM, anchor = 'w')

#create a label that shows the total "points recorded" (circles deleted)
delete_counter = Label (frame, text = "Total Points Recorded\n"+str(0), font = my_font)
delete_counter.pack(padx = 15, side = LEFT)

#set the maximum distance the path is allowed to be, create a label that shows the circles deleted subtracted from
#this distance in order to find the remaining distance
max_circ = 190
total_left = Label (frame, text = "Total Points Remaining\n"+str(max_circ), font = my_font)
total_left.pack(padx=15, side = LEFT)

#create a length scale
length_scale = Canvas (frame, width = cs*16, height = cs + 15)
length_scale.create_text((8*cs,cs-5), text = "10 points", font = my_font)
length_scale.create_line((3*cs, 2, 13*cs, 2), fill = "red", width = 5)
length_scale.create_line((3*cs,0,3*cs,int(3*cs/3.5)), fill = "red", width = 5)
length_scale.create_line((13*cs,0,13*cs,int(3*cs/3.5)), fill = "red", width = 5)
length_scale.pack(padx = 5, side = LEFT)

#create a save button that can be used to save the data path to file, and bind it to the save_callback method
save_button = Button (frame, text = "Save", width = 6, command = save_callback, font = my_font)
save_button.pack(padx=15, pady=2, side = RIGHT)

#bind the left mouse button to the callback method above, so the method is called when the mouse is clicked
w.bind("<Button-1>", callback)

mainloop()